<center><h1>UCI Diabetes Dataset</h1></center>

## Intro

The dataset represents 10 years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. It includes over 50 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria.
- (1)	It is an inpatient encounter (a hospital admission).
- (2)	It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.
- (3)	The length of stay was at least 1 day and at most 14 days.
- (4)	Laboratory tests were performed during the encounter.
- (5)	Medications were administered during the encounter.

The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc.

[Source](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008)

## Exploratory Data Analysis (EDA)

### Libraries

In [1]:
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sklearn.ensemble as se
import sklearn.feature_selection as fs
import sklearn.impute as si
import sklearn.linear_model as lm
import sklearn.metrics as sm
import sklearn.model_selection as ms
import xgboost as xg
from itertools import combinations
from tqdm import tqdm

### Data Loading

In [2]:
data = pd.read_csv('diabetes.csv')
data.head()

,id,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,1,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,...,No,No,No,No,No,No,No,No,No,NO
1,2,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,3,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,...,No,No,No,No,No,No,No,No,Yes,NO
3,4,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,5,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


### Counts

In [3]:
print('Number of Samples:', data.shape[0])
print('Number of Features:', data.shape[1])

Number of Samples: 101766
Number of Features: 51


In [4]:
data.describe()

,id,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,101766.000000,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,50883.500000,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,29377.458084,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.000000,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,25442.250000,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,50883.500000,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,76324.750000,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,101766.000000,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [5]:
print('Unique Individuals/Patients:', data.patient_nbr.nunique())
print('Number of Unique Encounters:', data.encounter_id.nunique())

Unique Individuals/Patients: 71518
Number of Unique Encounters: 101766


### Missingness

In [6]:
# Define a function to calculate the percentage of null values in a column
def null_percentage(col):
    possible_nulls = [None, np.nan, 'None', '?']
    null_count = col.isin(possible_nulls).sum()
    total_count = len(col)
    return round(null_count / total_count * 100)

# Calculate the percentage of null values in each column
data.apply(null_percentage)

id                           0
encounter_id                 0
patient_nbr                  0
race                         2
gender                       0
age                          0
weight                      97
admission_type_id            0
discharge_disposition_id     0
admission_source_id          0
time_in_hospital             0
payer_code                  40
medical_specialty           49
num_lab_procedures           0
num_procedures               0
num_medications              0
number_outpatient            0
number_emergency             0
number_inpatient             0
diag_1                       0
diag_2                       0
diag_3                       1
number_diagnoses             0
max_glu_serum               95
A1Cresult                   83
metformin                    0
repaglinide                  0
nateglinide                  0
chlorpropamide               0
glimepiride                  0
acetohexamide                0
glipizide                    0
glyburid

Some features have more missing values than other.

 - weight
 - max_glu_serum
 - A1Cresult

Have the highest percentage of null values.

### Near-Zero Variance Features

In [7]:
# Define a threshold for near-zero variance
threshold = 0.0

# Encode categorical features as integers
df_encoded = data.select_dtypes(include=['object']).apply(lambda x: pd.factorize(x)[0])
df_encoded = pd.concat([df_encoded, data.select_dtypes(include=['int', 'float'])], axis=1)

# Instantiate VarianceThreshold object
selector = fs.VarianceThreshold(threshold)

# Fit selector to DataFrame
selector.fit(df_encoded)

# Get boolean mask of features that meet threshold
mask = selector.get_support()

# Get list of column names that meet threshold
near_zero_var_cols = data.columns[~mask].tolist()

# Print the result
print('Columns with near-zero variance:', near_zero_var_cols)

Columns with near-zero variance: ['repaglinide', 'nateglinide']


Two features (repaglinide and nateglinide) have no variance, meaning that each patient has the same value. We can then remove these features as they don't have any information.

## Data Preparation

### Data Dictionary

 - `id` The id of the row, simply to identify the entry in the db.
 - `encounter_id` The unique id for each encounter/visit with a doctor. e.g. every time that a patient goes to a hospital, it is a different encounter.
 - `patient_nbr` The patient identifier.
 - `race` The patient's ethnicity.
 - `gender` The patient's gender.
 - `age` The patient's age.
 - `weight` The patient's weight.
 - `admission_type_id` An identifier for the type of admission, e.g. hospital, emergency room, surgery, etc. (You're not entirely sure which one is which but it's ok for now).
 - `discharge_disposition_id` An identifier for the type of disharge, e.g. hospital, home, emergency room, surgery, etc. (You're not entirely sure which one is which but it's ok for now).
 - `admission_source_id` An identifier for the type of admission source. (You're not entirely sure which one is which but it's ok for now).
 - `time_in_hospital` The number of days spent in hospital.
 - `payer_code` The code of that patient's insurance.
 - `medical_specialty` The type of medical specialty related to that person's visit.
 - `num_lab_procedures` The number of lab procedures performed on that patient.
 - `num_procedures` The number of procedures/surgeries performed on that patient.
 - `num_medications` The number of medications/drugs that that person is on.
 - `number_outpatient` The number of outpatient visits.
 - `number_emergency` The number of emergency room visits.
 - `number_inpatient` The number of inpatient visits.
 - `diag_1` The primary diagnosis.
 - `diag_2` The secondary diagnosis.
 - `diag_3` The tertiary diagnosis.
 - `number_diagnoses` The total number of diagnoses.
 - `max_glu_serum` The patient's blood glucose maximum value.
 - `A1Cresult` A biomarker that indicates a diabetic's severity.
 - `metformin` Is that person on this drug? (Yes/No)
 - `repaglinide` Is that person on this drug? (Yes/No)
 - `nateglinide` Is that person on this drug? (Yes/No)
 - `chlorpropamide` Is that person on this drug? (Yes/No)
 - `glimepiride` Is that person on this drug? (Yes/No)
 - `acetohexamide` Is that person on this drug? (Yes/No)
 - `glipizide` Is that person on this drug? (Yes/No)
 - `glyburide` Is that person on this drug? (Yes/No)
 - `tolbutamide` Is that person on this drug? (Yes/No)
 - `pioglitazone` Is that person on this drug? (Yes/No)
 - `rosiglitazone` Is that person on this drug? (Yes/No)
 - `acarbose` Is that person on this drug? (Yes/No)
 - `miglitol` Is that person on this drug? (Yes/No)
 - `troglitazone` Is that person on this drug? (Yes/No)
 - `tolazamide` Is that person on this drug? (Yes/No)
 - `examide` Is that person on this drug? (Yes/No)
 - `citoglipton` Is that person on this drug? (Yes/No)
 - `insulin` Is that person on this drug? (Yes/No)
 - `glyburide.metformin` Is that person on this drug? (Yes/No)
 - `glipizide.metformin` Is that person on this drug? (Yes/No)
 - `glimepiride.pioglitazone` Is that person on this drug? (Yes/No)
 - `metformin.rosiglitazone` Is that person on this drug? (Yes/No)
 - `metformin.pioglitazone` Is that person on this drug? (Yes/No)
 - `change` The change in HbA1c.
 - `diabetesMed` Whether the patient is on a diabetes medication.
 - `readmitted` Whether the visit is a readmission.

### Parameters

In [8]:
non_predictive_features = ['id', 'encounter_id', 'patient_nbr']
categorical_features = ['race', 'gender', 'admission_type_id',
                        'discharge_disposition_id', 'admission_source_id',
                        'payer_code', 'medical_specialty',
                        'diag_1', 'diag_2', 'diag_3',
                        'diag_1_category', 'diag_2_category', 'diag_3_category']
ex_post_features = []
target = 'A1Cresult'

### Data Cleaning

In [9]:
def clean_data(df):
    df = df.copy()

    for col in df.columns:
        # Replace '?' with None
        df[col] = df[col].apply(lambda x: None if x == "?" else x)

        # Convert 'NO', '<30', and '>30' to 0 and 1 respectively
        if set(df[col].unique()) == {'NO', '<30', '>30'}:
            df[col] = df[col].apply(lambda x: 0 if x == 'NO' else 1)

        # Check the max_glu_serum column
        if col == 'max_glu_serum':
            df[col] = df[col].apply(lambda x: None if x == 'None' else (301 if x == '>300' else (85 if x == 'Norm' else (201 if x == '>200' else x))))

        # Convert '>xxx' and '<xxx' to just 'xxx'
        df[col] = df[col].apply(lambda x: float(x[1:]) + 1 if isinstance(x, str) and x.startswith(">") else x)
        df[col] = df[col].apply(lambda x: float(x[1:]) - 1 if isinstance(x, str) and x.startswith("<") else x)

        # Convert '[xx-yy)' to the average of xx and yy
        df[col] = df[col].apply(lambda x: (float(x.split("-")[0][1:]) + float(x.split("-")[1][:-1]))/2 if isinstance(x, str) and x.startswith("[") and x.endswith(")") else x)

        # Convert 'No' to 0, 'Steady', 'Up', and 'Down' to 1
        if set(df[col].unique()).issubset({'No', 'Steady', 'Up', 'Down'}):
            df[col] = df[col].apply(lambda x: 1 if x in {'Steady', 'Up', 'Down'} else 0)

        # Convert 'No' and 'Ch' or 'No' and 'Yes' to 0 and 1 respectively
        elif set(df[col].unique()) == {'No', 'Ch'} or set(df[col].unique()) == {'No', 'Yes'}:
            df[col] = df[col].apply(lambda x: 1 if x in {'Yes', 'Ch'} else 0 if x == 'No' else x)

        # Convert 'None' to None
        df[col] = df[col].apply(lambda x: None if x == "None" else x)

        # Convert 'NO' to 0
        df[col] = df[col].apply(lambda x: 0 if x == 'NO' else x)

    return df

cleaned = clean_data(data)

### Data Aggregation

We want to make a model that takes the current characteristics of a patient and estimates their current HbA1c. For simplicity's sake, we take the latest encounter, which we consider the patient's current characteristics and HbA1c. We are leaving information on the table from all the previous encounters but we approach the model as an MVP (Minimum Viable Product), meaning that we develop a simple model first and we can then go back and add new features using the past information.

In [10]:
individuals = cleaned.loc[cleaned.reset_index().groupby(['patient_nbr'])['encounter_id'].idxmax()]

### Feature Engineering

In [11]:
def map_icd_codes(code):
  if code is None:
    return 'Unknown'
  elif code[:1] in ['E', 'V']:
    return 'Other'
  else:
    code = float(code)
    if (code >= 390 and code <= 459) or code == 785:
        return 'Circulatory'
    elif (code >= 460 and code <= 519) or code == 786:
        return 'Respiratory'
    elif (code >= 520 and code <= 579) or code == 787:
        return 'Digestive'
    elif (code >= 250 and code < 251):
        return 'Diabetes'
    elif code >= 800 and code <= 999:
        return 'Injury'
    elif code >= 710 and code <= 739:
        return 'Musculoskeletal'
    elif (code >= 580 and code <= 629) or code == 788:
        return 'Genitourinary'
    elif code >= 140 and code <= 239:
        return 'Neoplasms'
    elif code in ['780', '781', '784', '790', '791', '792', '793', '794', '795', '796', '797', '798', '799'] \
     or (code >= 1 and code <= 139) \
     or (code >= 771 and code <= 779) \
     or (code >= 290 and code <= 319) \
     or (code >= 306 and code <= 316) \
     or (code >= 320 and code <= 359) \
     or (code == 7805) \
     or (code >= 1 and code <= 999 and code < 140) \
     or (code >= 630 and code <= 679) \
     or (code >= 680 and code <= 709) \
     or (code >= 280 and code <= 289) \
     or (code >= 360 and code <= 389) \
     or (code >= 740 and code <= 759):
        return 'Other'
    else:
        return 'Unknown'

# Apply the function to the diag_1 column
individuals['diag_1_category'] = individuals['diag_1'].apply(map_icd_codes)
individuals['diag_2_category'] = individuals['diag_2'].apply(map_icd_codes)
individuals['diag_3_category'] = individuals['diag_3'].apply(map_icd_codes)

In [12]:
individuals.drop('diag_1', axis=1, inplace=True)
individuals.drop('diag_2', axis=1, inplace=True)
individuals.drop('diag_3', axis=1, inplace=True)

#### One-Hot Encoding

One-Hot Encoding to convert the categorical features into binary ones.

In [13]:
for c in categorical_features:
    if c in individuals.columns:
        one_hot = pd.get_dummies(individuals[c], prefix=c)
        individuals = individuals.drop(c, axis=1)
        individuals = individuals.join(one_hot)

One of the issues with One-Hot Encoding is that it increases the dimensionality of the dataset (meaning that we have sometimes way more features than we had before), creating what's called the Curse of Dimensionality.

In [14]:
print('Number of Features:', individuals.shape[1])

Number of Features: 216


In this case, we went from 51 features to 216. That's a large increase and lots of new features. There are other ways to turn categorical features into numerical ones, such as Target Encoding, but because they make the model more difficult to interpret, we stick with One-Hot Encoding as we still have a ratio of samples to features of ~4 to 1.

If we had kept the diagnoses as individual values, without grouping them, we would have had thousands of new features.

#### Non-Predictive Features

Removing the non-predictive features, such as the IDs and the Diabetes category, as everybody in this dataset has Diabetes.

In [15]:
individuals = individuals.drop(non_predictive_features, axis=1)

In [16]:
individuals.drop('diag_1_category_Diabetes', axis=1, inplace=True)
individuals.drop('diag_2_category_Diabetes', axis=1, inplace=True)
individuals.drop('diag_3_category_Diabetes', axis=1, inplace=True)

### Train, Validation and Test Split

There are three types of individuals based on HbA1c:
 1. Those that have HbA1c on file, and it is over 8
 2. Those that have HbA1c on file, and it is under 8
 3. Those that do not have HbA1c on file, meaning that information is missing

Our model is going to be trained on the data we have information for (1. and 2.) and we will use it to then estimate the HbA1c's level (>8 or \<8) for the unknown population (3.).

In [17]:
train_test = individuals[~individuals['A1Cresult'].isna()]
unknown_population = individuals[individuals['A1Cresult'].isna()]
print('Known Population:', train_test.shape[0])
print('Unknown Population:', unknown_population.shape[0])
print('Total Population:', train_test.shape[0] + unknown_population.shape[0])

Known Population: 12398
Unknown Population: 59120
Total Population: 71518


In [18]:
X = individuals[individuals.columns[~individuals.columns.isin(['A1Cresult'])]]
y = np.where(individuals['A1Cresult'] == 9.0, 1, 0)

train_X, test_X, train_y, test_y = ms.train_test_split(X, y)

### Imputation

Usually, Mean Imputation is a pretty bad form of imputation. It replaces every unknown value with the mean which is, almost always, not a very representative metric. Better forms of imputation, such as KNN Imputation, use Machine Learning to impute the missing data. In this case, because we are developing the first model, we use Mean Imputation as a placeholder. It is a good module to keep and we can later on replace it with a better approach.

It's important to do imputation after having split Training and Test sets, as you cannot use the Test set's information to impute the Training set's data (otherwise it counts as data leakage).

In [19]:
imputer = si.SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(train_X[['weight']])

train_X['weight'] = imputer.transform(train_X[['weight']])
test_X['weight'] = imputer.transform(test_X[['weight']])

In [20]:
imputer = si.SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(train_X[['max_glu_serum']])

train_X['max_glu_serum'] = imputer.transform(train_X[['max_glu_serum']])
test_X['max_glu_serum'] = imputer.transform(test_X[['max_glu_serum']])

### Saving Data

In [21]:
with open('train_X.csv', 'w') as FOUT:
    np.savetxt(FOUT, train_X)

with open('test_X.csv', 'w') as FOUT:
    np.savetxt(FOUT, test_X)

with open('train_y.csv', 'w') as FOUT:
    np.savetxt(FOUT, train_y)

with open('test_y.csv', 'w') as FOUT:
    np.savetxt(FOUT, test_y)
    
with open('features.pkl', 'wb') as f:
    pickle.dump(X.columns, f)